In [2]:
import numpy as np

In [17]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))


def sigmoid_derivative(z):
    return sigmoid(z)*(1-sigmoid(z))


def tanh(z):
    return np.tanh(z)


def tanh_derivative(z):
    return 1 - np.square(tanh(z))


def relu(z):
    return np.maximum(z, 0)


def relu_derivative(z):
    return int(z >= 0)

ACTIVATIONS = {'sigmoid': sigmoid,
               'tanh': tanh,
               'relu': relu}
DERIVATIVES = {'sigmoid': sigmoid_derivative,
               'tanh': tanh_derivative,
               'relu': relu_derivative}

In [8]:
class MultilayerPerceptron(object):

    def __init__(self, layers, activation_functions):
        self.layers_count = len(layers)
        self.layers = layers
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(layers[:-1], layers[1:])]
        self.biases = [np.random.randn(y, 1) for y in layers[1:]]

    def feedforward(self):

In [16]:
mp = MultilayerPerceptron([2, 3, 1], None)

1